# Wrappers example

We will see on this notebook what are the wrappers defined by Sinergym and how to use them. You can also implement your own wrappers inheriting from *gym.Wrapper* or some of its variants.

In [1]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import *

## Multi-Objective Wrapper

[MO-Gymnasium](https://github.com/Farama-Foundation/MO-Gymnasium) is an open source Python library for developing and comparing multi-objective reinforcement learning algorithms. These environments return a reward vector instead of a scalar value; one for each objective.

In order to be more general as possible, it could be interesting that Sinergym would give that reward vector too. In this way, Sinergym would have compatibility with both; multi-objective algorithms and algorithms that work with a traditional reward value.

We can transform reward returned in a vector using the next wrapper:

In [2]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env=MultiObjectiveReward(env,reward_terms=['reward_energy','reward_comfort'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.
[WRAPPER MultiObjectiveReward] (INFO) : wrapper initialized.


You have to ensure that `reward_terms` are available in `info` dict returned in step method of the environment. Otherwise, we will encounter an execution error.
By default, Sinergym environments return in `info` dict all reward terms specified in reward class used, so if the objective exists in reward term you shouldn't have any problem.

In [3]:
env.reset()
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
env.close()
print(reward)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/5ZoneAutoDXVAV.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
[-0.11085250479199188, -2.8562402567058562]


## Previous Observations Wrappers

This Wrapper will add observation values from previous timestep to current environment observation.
It is possible to select the variables you want to track its previous observation values. 
Observation space will be updated with the new dimension.

In [4]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = PreviousObservationWrapper(env, previous_variables=[
        'htg_setpoint',
        'clg_setpoint',
        'air_temperature'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res2]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.
[WRAPPER PreviousObservationWrapper] (INFO) : Wrapper initialized.


We can see the observation values has been updated:

In [5]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.observation_variables,obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res2/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res2/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Ep

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
NEW OBSERVATION:  {'month': 1.0, 'day_of_month': 1.0, 'hour': 0.0, 'outdoor_temperature': 4.8, 'outdoor_humidity': 61.0, 'wind_speed': 4.65, 'wind_direction': 160.0, 'diffuse_solar_radiation': 0.0, 'direct_solar_radiation': 0.0, 'htg_setpoint': 12.8, 'clg_setpoint': 40.0, 'air_temperature': 17.14376, 'air_humidity': 28.904009, 'people_occupant': 0.0, 'co2_emission': 0.0, 'HVAC_electricity_demand_rate': 1108.525, 'total_electricity_HVAC': 997672.56, 'htg_setpoint_previous': 12.8, 'clg_setpoint_previous': 40.0, 'air_temperature_previous': 17.248121}


## Datetime Wrapper

This wrapper will substitute `day` value by `is_weekend` flag, and `hour` and `month` by sin and cos values. 
Observation space is also updated automatically.

In [6]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = DatetimeWrapper(env)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res3]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.
[WRAPPER DatetimeWrapper] (INFO) : Wrapper initialized.


Concretely, this wrapper deletes the observation variables `month`, `day` and `hour`, and adds `month_sin`, `month_cos`, `is_weekend`, `hour_sin` and `hour_cos` instead:

In [7]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.observation_variables,obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res3/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res3/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Ep

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
NEW OBSERVATION:  {'month_cos': 1.0, 'month_sin': 0.0, 'is_weekend': 0.0, 'hour_cos': 1.0, 'hour_sin': 0.0, 'outdoor_temperature': 4.800000190734863, 'outdoor_humidity': 61.0, 'wind_speed': 4.650000095367432, 'wind_direction': 160.0, 'diffuse_solar_radiation': 0.0, 'direct_solar_radiation': 0.0, 'htg_setpoint': 12.800000190734863, 'clg_setpoint': 40.0, 'air_temperature': 17.143760681152344, 'air_humidity': 28.904008865356445, 'people_occupant': 0.0, 'co2_emission': 0.0, 'HVAC_electricity_demand_rate': 1108.5250244140625, 'total_electricity_HVAC': 997672.5625}


## Discrete Incremental Wrapper

A wrapper for an incremental setpoint action space environment. This wrapper
  will update a *continuous* environment, converting it in a *discrete* environment with an action mapping and action space 
  depending on the step and delta specified. The action will be sum with current setpoint values instead of overwrite the latest action. 
  Then, the action is the current setpoints values with the increase instead of the index of the action_mapping for the continuous 
  environment in the next layer.

In [8]:
env=gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ',env.action_space)
env = DiscreteIncrementalWrapper(
        env,initial_values=[21.0,25.0], delta_temp=2, step_temp=0.5)
print('WRAPPED ACTION SPACE: ',env.action_space)
print('WRAPPED ACTION MAPPING: ',env.action_mapping)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-continuous-v1 created successfully.
ORIGINAL ACTION SPACE:  Box(-1.0, 1.0, (2,), float32)
[WRAPPER DiscreteIncrementalWrapper] (INFO) : New incremental actio

Maximum and minimum values to create the action mapping is read from environment action space, so that the setpoint increments and decrements do not go outside the agreed limits.
The delta and step values are used to determine how the discrete space of these increments and decrements will be constructed.
An example of how it works is shown below:

In [9]:
env.reset()
print('CURRENT SETPOINTS VALUES: ', env.current_setpoints)
for i in range(5):
    action=env.action_space.sample()
    _,_,_,_,info=env.step(action)
    print('Action number ',i,': ',env.action_mapping[action])
    print('Setpoints update: ', info['action'])
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examp

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
CURRENT SETPOINTS VALUES:  [21.0, 25.0]
Action number  0 :  [1.5, 0.0]
Setpoints update:  [22.5, 25.0]
Action number  1 :  [-0.5, 0.0]
Setpoints update:  [22.0, 25.0]
Action number  2 :  [0.0, 0.0]
Setpoints update:  [22.0, 25.0]
Action number  3 :  [0.0, 1.5]
Setpoints update:  [22.0, 26.5]
Action number  4 :  [-1.5, 0.0]
Setpoints update:  [20.5, 26.5]


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-v1]


## Normalization Wrapper

This wrapper is used to transform observation received from simulator in values between -1 and 1.
It is based in the [dynamic normalization wrapper of Gymnasium](https://gymnasium.farama.org/_modules/gymnasium/wrappers/normalize/#NormalizeObservation). At the beginning,
it is not precise and the values may be out of range usually, so use this wrapper carefully.

In [10]:
#Original env
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = NormalizeObservation(
        env=env)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res4]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.


We can see in the next code how the variables specified have been normalized correctly:

In [11]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.observation_variables,obs))
env.close()
print('OBSERVATION WITH NORMALIZATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res4/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res4/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Ep

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
OBSERVATION WITH NORMALIZATION IN VARIABLES SELECTED:  {'month': 1.0, 'day_of_month': 1.0, 'hour': 0.0, 'outdoor_temperature': 4.8, 'outdoor_humidity': 61.0, 'wind_speed': 4.65, 'wind_direction': 160.0, 'diffuse_solar_radiation': 0.0, 'direct_solar_radiation': 0.0, 'htg_setpoint': 12.8, 'clg_setpoint': 40.0, 'air_temperature': 17.14376, 'air_humidity': 28.904009, 'people_occupant': 0.0, 'co2_emission': 0.0, 'HVAC_electricity_demand_rate': 1108.525, 'total_electricity_HVAC': 997672.56}


## Logger Wrapper

Wrapper for logging all interactions between agent and environment. Logger class can be selected
in the constructor if other type of logging is required. For more information about *Sinergym* Logger visit [Logger](https://ugr-sail.github.io/sinergym/compilation/main/pages/output.html#logger)

In [12]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env=LoggerWrapper(env)


env.reset()    
terminated = False
current_month = 0
while not terminated:
    a = env.action_space.sample()
    _,_,terminated,_,_=env.step(a)
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res5]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.
#------------------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active
Progress: |***************************************************************************************************| 99%
[WRAPPER LoggerWrapper] (INFO) : End of episode, recording summary (progress.csv) if logger is active
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

## Multi Observation Wrapper

This wrapper will stack observation received in a history queue (size can be customized).

In [13]:
#Original environment
env=gym.make('Eplus-5zone-hot-discrete-v1')
obs, info=env.reset()
print('BEFORE MULTI OBSERVATION: ',obs)

#Multi Observation environment
env=MultiObsWrapper(env, n=5, flatten=True)
obs, info=env.reset()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res6]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-discrete-v1 created successfully.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Startin

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
BEFORE MULTI OBSERVATION:  [1.0000000e+00 1.0000000e+00 0.0000000e+00 4.4000001e+00 6.5000000e+01
 3.8750000e+00 1.4500000e+02 0.0000000e+00 0.0000000e+00 1.2800000e+01
 4.0000000e+01 1.7248121e+01 2.8887291e+01 0.0000000e+00 0.0000000e+00
 1.0360292e+03 9.3242619e+05]
[WRAPPER MultiObsWrapper] (INFO) : Wrapper initialized.
Progress: |***************************************************************************************************| 99%
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 2]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res6/Eplus-env-sub_run2]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TM

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.


In [14]:
print('AFTER MULTI OBSERVATION: ',obs)
env.close()

AFTER MULTI OBSERVATION:  [1.0000000e+00 1.0000000e+00 0.0000000e+00 4.4000001e+00 6.5000000e+01
 3.8750000e+00 1.4500000e+02 0.0000000e+00 0.0000000e+00 1.2800000e+01
 4.0000000e+01 1.7248121e+01 2.8887291e+01 0.0000000e+00 0.0000000e+00
 1.0360292e+03 9.3242619e+05 1.0000000e+00 1.0000000e+00 0.0000000e+00
 4.4000001e+00 6.5000000e+01 3.8750000e+00 1.4500000e+02 0.0000000e+00
 0.0000000e+00 1.2800000e+01 4.0000000e+01 1.7248121e+01 2.8887291e+01
 0.0000000e+00 0.0000000e+00 1.0360292e+03 9.3242619e+05 1.0000000e+00
 1.0000000e+00 0.0000000e+00 4.4000001e+00 6.5000000e+01 3.8750000e+00
 1.4500000e+02 0.0000000e+00 0.0000000e+00 1.2800000e+01 4.0000000e+01
 1.7248121e+01 2.8887291e+01 0.0000000e+00 0.0000000e+00 1.0360292e+03
 9.3242619e+05 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.4000001e+00
 6.5000000e+01 3.8750000e+00 1.4500000e+02 0.0000000e+00 0.0000000e+00
 1.2800000e+01 4.0000000e+01 1.7248121e+01 2.8887291e+01 0.0000000e+00
 0.0000000e+00 1.0360292e+03 9.3242619e+05 1.000000

Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]


## Nesting wrappers

All the wrappers listed in this notebook can be nested in order to combine functionalities added. However, **the order** in which these wrappers are nested is critical to their proper functionality. We strongly recommend that it be in the same order in which they appear here or in the documentation. In case you want to nest a subset of the total, it is not a problem as long as you continue respecting this order.
We see and example:

In [15]:
env = gym.make('Eplus-5zone-hot-continuous-v1')
env = MultiObjectiveReward(
    env=env,
    reward_terms=[
        'reward_energy',
        'reward_comfort'])
env = PreviousObservationWrapper(env, previous_variables=[
    'htg_setpoint',
    'clg_setpoint',
    'air_temperature'])
env = DatetimeWrapper(env)
env = DiscreteIncrementalWrapper(
    env,initial_values=[21.0,25.0], delta_temp=2, step_temp=0.5)
env = NormalizeObservation(
    env=env,
    ranges=RANGES_5ZONE,
    variables=[
        'htg_setpoint',
        'clg_setpoint',
        'air_temperature',
        'htg_setpoint_previous',
        'clg_setpoint_previous',
        'air_temperature_previous'])
env = LoggerWrapper(env=env, flag=True)
env = MultiObsWrapper(env=env, n=5, flatten=True)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res2]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-continuous-v1 created successfully.
[WRAPPER MultiObjectiveReward] (INFO) : wrapper initialized.
[WRAPPER PreviousObservationWrapper] (INFO) : Wrapper initia

Now we just simply use the environment with the wrappers, for example:

In [16]:
for i in range(1):
    obs, info = env.reset()
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', reward, info)
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res2/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res2/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examp

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active
Reward:  [-0.11085250479199188, -2.8562402567058562] {'time_elapsed(hours)': 0.5, 'month': 1, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [21.0, 23.0], 'timestep': 2, 'reward': -1.483546380748924, 'reward_energy': -0.11085250479199188, 'reward_comfort': -2.8562402567058562, 'total_energy': 1108.5250479199187, 'abs_comfort': 2.8562402567058562, 'temperatures': [17.143759743294144]}


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Reward:  [-0.01030338481821307, -0.0] {'time_elapsed(hours)': 744.375, 'month': 2, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [19.0, 28.5], 'timestep': 2977, 'reward': -0.005151692409106535, 'reward_energy': -0.01030338481821307, 'reward_comfort': -0.0, 'total_energy': 103.0338481821307, 'abs_comfort': 0.0, 'temperatures': [20.326224703060248]}
Reward:  [-0.013657983341870036, -0.0] {'time_elapsed(hours)': 1416.375, 'month': 3, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [20.5, 22.5], 'timestep': 5665, 'reward': -0.006828991670935018, 'reward_energy': -0.013657983341870036, 'reward_comfort': -0.0, 'total_energy': 136.57983341870036, 'abs_comfort': 0.0, 'temperatures': [20.500028024178043]}
Reward:  [-0.0, -0.0] {'time_elapsed(hours)': 2160.25, 'month': 4, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [20.0, 28.0], 'timestep': 8641, 'reward': -0.0, 'reward_energy': -0.0, 'reward_comfort': -0.0, 'total_energy': 0.0, 'abs_comfort': 0.0, 'temperatures': [20.803695489